In [1]:
# For an interactive backend
%matplotlib inline

# To replace the reference to __file__ in the .py files
import os
GRATING_DIR_=os.getcwd()

In [ ]:
""" Copyright chriskeraly
    Copyright (c) 2019 Lumerical Inc. """

######## IMPORTS ########
# General purpose imports
import os
import numpy as np

# Optimization specific imports
from lumopt.utilities.load_lumerical_scripts import load_from_lsf
from lumopt.utilities.wavelengths import Wavelengths
from lumopt.utilities.materials import Material
from lumopt.geometries.polygon import FunctionDefinedPolygon
from lumopt.figures_of_merit.modematch import ModeMatch
from lumopt.optimizers.generic_optimizers import ScipyOptimizers
from lumopt.optimization import Optimization

######## DEFINE BASE SIMULATION ########
script = load_from_lsf(os.path.join(GRATING_DIR_, 'grating_base.lsf'))

######## DEFINE SPECTRAL RANGE #########
wavelengths = Wavelengths(start = 1300e-9, stop = 1800e-9, points = 21)

######## DEFINE MATERIALS ##############
oxide = Material(1.44**2, mesh_order = 1)
silicon = Material(3.4**2)

######## DEFINE OPTIMIZABLE GEOMETRY ########
n_grates = 22

def grate_function_generator(grate_number, grate_height = 70e-9, grate_center_y = 220e-9-35e-9):
    def grate_function(params):
        grate_position = params[grate_number]
        grate_width = params[int(grate_number + len(params)/2)]
        left = grate_position - grate_width/2.0
        right = grate_position + grate_width/2.0
        top = grate_center_y + grate_height/2.0
        bottom = grate_center_y - grate_height/2.0
        polygon_points_x = [left, left, right, right]
        polygon_points_y = [top, bottom, bottom, top]
        polygon_points=[(x, y) for x, y in zip(polygon_points_x, polygon_points_y)]
        return np.array(polygon_points)
    return grate_function

initial_positions = 550e-9*np.arange(n_grates)+300e-9
initial_widths = 100e-9*np.ones(n_grates)
edge_precision = 20
bounds = [(250e-9, 15e-6)]*n_grates+[(100e-9,500e-9)]*n_grates
initial_params = np.concatenate((initial_positions,initial_widths))
first_tooth = FunctionDefinedPolygon(func = grate_function_generator(0), initial_params = initial_params, bounds = bounds, z = 0.0, depth = 1e-6, eps_out = silicon, eps_in = oxide, edge_precision = edge_precision, dx = 0.1e-9)
full_geometry = first_tooth
for i in range(1,n_grates):
    new_tooth = FunctionDefinedPolygon(func = grate_function_generator(i), initial_params = initial_params, bounds=bounds, z = 0.0, depth = 1e-6, eps_out = silicon, eps_in = oxide, edge_precision = edge_precision, dx = 0.1e-9)
    full_geometry = full_geometry*new_tooth

######## DEFINE FIGURE OF MERIT ########
fom = ModeMatch(monitor_name = 'fom', mode_number = 1, direction = 'Forward', target_T_fwd = lambda wl: np.ones(wl.size), norm_p = 1)

######## DEFINE OPTIMIZATION ALGORITHM ########
optimizer = ScipyOptimizers(max_iter = 30, method = 'L-BFGS-B', scaling_factor = 1e6, pgtol = 1e-9)

######## PUT EVERYTHING TOGETHER ########
opt = Optimization(base_script = script, wavelengths = wavelengths, fom = fom, geometry = full_geometry, optimizer = optimizer, hide_fdtd_cad = False, use_deps = True)

######## RUN THE OPTIMIZER ########
opt.run()

CONFIGURATION FILE {'root': 'c:\\users\\user\\documents\\research\\lumopt', 'lumapi': '\\Program Files\\Lumerical\\FDTD\\api\\python'}
Accurate interface detection enabled


C:\Users\User\Miniconda3\envs\LumTest\lib\site-packages\matplotlib-3.0.3-py3.7-win-amd64.egg\matplotlib\figure.py:445: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


Running scipy optimizer
bounds= [[ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.25 15.  ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]
 [ 0.1   0.5 ]]
start= [ 0.3   0.85  1.4   1.95  2.5   3.05  3.6   4.15  4.7   5.25  5.8   6.35
  6.9   7.45  8.    8.55  9.1   9.65 10.2  10.75 11.3  11.85  0.1   0.1
  0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1
  0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1 ]
Running forward solves
FOM = 0.02